# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846832256807                   -0.70    4.5
  2   -7.852326789838       -2.26       -1.53    1.0
  3   -7.852646028740       -3.50       -2.52    3.0
  4   -7.852646676655       -6.19       -3.36    2.2
  5   -7.852646685804       -8.04       -4.79    1.2
  6   -7.852646686725       -9.04       -5.17    4.2
  7   -7.852646686729      -11.35       -5.83    2.0
  8   -7.852646686730      -12.37       -7.31    2.0
  9   -7.852646686730   +  -14.75       -7.92    2.8
 10   -7.852646686730      -14.75       -8.66    2.0
 11   -7.852646686730   +    -Inf       -9.14    1.0
 12   -7.852646686730      -14.75      -10.48    3.0
 13   -7.852646686730   +  -14.35      -11.66    2.2
 14   -7.852646686730      -14.35      -11.67    1.2
 15   -7.852646686730   +    -Inf      -12.93    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846820022777                   -0.70           4.8
  2   -7.852526806182       -2.24       -1.63   0.80    2.0
  3   -7.852636745594       -3.96       -2.72   0.80    1.0
  4   -7.852646517437       -5.01       -3.29   0.80    2.2
  5   -7.852646682965       -6.78       -4.15   0.80    1.2
  6   -7.852646686420       -8.46       -4.90   0.80    1.2
  7   -7.852646686724       -9.52       -5.83   0.80    2.0
  8   -7.852646686730      -11.24       -6.71   0.80    2.0
  9   -7.852646686730      -12.81       -7.56   0.80    2.5
 10   -7.852646686730   +  -15.05       -8.37   0.80    1.8
 11   -7.852646686730      -15.05       -8.41   0.80    2.2
 12   -7.852646686730   +    -Inf      -10.01   0.80    1.0
 13   -7.852646686730   +    -Inf      -10.62   0.80    2.8
 14   -7.852646686730      -14.75      -11.05   0.80    1.2
 15   -7.852646686730   +  -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.399177e+01     4.253909e+00
 * time: 0.40346503257751465
     1     1.381368e+00     1.949028e+00
 * time: 0.6069169044494629
     2    -1.451453e+00     2.406721e+00
 * time: 0.631119966506958
     3    -3.521434e+00     1.916651e+00
 * time: 0.6665189266204834
     4    -4.783729e+00     1.842281e+00
 * time: 0.7024519443511963
     5    -6.556500e+00     1.027800e+00
 * time: 0.738663911819458
     6    -7.302109e+00     5.114658e-01
 * time: 0.7741148471832275
     7    -7.589990e+00     4.196157e-01
 * time: 0.7984328269958496
     8    -7.704515e+00     1.742432e-01
 * time: 0.8227479457855225
     9    -7.780402e+00     1.913568e-01
 * time: 0.8469409942626953
    10    -7.813197e+00     7.128470e-02
 * time: 0.8712449073791504
    11    -7.834465e+00     7.093862e-02
 * time: 0.8954908847808838
    12    -7.844443e+00     5.368028e-02
 * time: 0.919625997543335
    13    -7.847666e+00     3.707026e-02
 * time: 0.943791866302

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846681515229                   -0.70    4.5
  2   -7.852294638446       -2.25       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686709                   -2.54
  2   -7.852646686730      -10.67       -5.94
  3   -7.852646686730   +  -14.75      -12.85


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.0221261220441639e-13
|ρ_newton - ρ_scfv| = 2.7143195103018e-13
|ρ_newton - ρ_dm|   = 3.072497108365577e-10
